# Installation des extensions requises

Il est important d'inclure le le fichier requierement.txt
dans votre repertoire de travail soit indiquer le
chemin qui y mene vers le fichier requirement.txt.
Une fois que le chemin est bien renseigner, exécuter la commande suivante pour l'installation des bibliothèques

In [ ]:
pip install -r requirements.txt

# Import des bibliothèques

In [71]:
import os
import sys
import csv
import ntpath
# import nilearn
import numpy as np
import pandas as pd
import nibabel as nib
from glob import glob
import SimpleITK as sitk
import nibabel.processing
from radiomics import featureextractor
from tqdm import tqdm_notebook as tqdm

# Chemin vers les données

Les données sont zipé. Indiquer le chemin complet jusqu'aux données. Le dossier lesions comporte les images et masques avec des pondérations T2w et diffusion ADC en plus des données clinic

In [130]:
root_dir = "/home/sama/Documents/Stage_CIC/Stage-Github/lesions/"

clin_info = os.path.join(root_dir, "PROSTATEx_Classes.csv")

In [131]:
df = pd.read_csv(clin_info)

# Visualisation des données du dataset

In [132]:
df.head(5)

,ID,Clinically Significant,Gleason_Grade_Group
0,ProstateX-0000_Finding1,True,3
1,ProstateX-0001_Finding1,False,1
2,ProstateX-0002_Finding1,True,2
3,ProstateX-0002_Finding2,False,1
4,ProstateX-0003_Finding1,False,No biopsy information


# Taille du dataset

In [133]:
len(df)

299

# Suppression des données sans informations de biopsie

In [134]:
df_clif = pd.read_csv(clin_info) # Permet de regrouper les données en fonction de la colonne des infos clinic

df_clif.drop(df_clif[df_clif['Gleason_Grade_Group']=='No biopsy information'].index, inplace=True)
df_clif.drop(df_clif[df_clif['ID']=='ProstateX-0028_Finding1'].index, inplace = True)
df_clif.drop(df_clif[df_clif['ID']=='ProstateX-0031_Finding1'].index, inplace = True)
df_clif.drop(df_clif[df_clif['ID']=='ProstateX-0031_Finding2'].index, inplace = True)
df_clif.drop(df_clif[df_clif['ID']=='ProstateX-0128_Finding1'].index, inplace = True)
# df_clif.drop(df_clif[df_clif['ID']=='ProstateX-0002_Finding1'].index, inplace = True)
# df_clif.drop(df_clif[df_clif['ID']=='ProstateX-0005_Finding1'].index, inplace = True)
# df_clif.drop(df_clif[df_clif['ID']=='ProstateX-0014_Finding1'].index, inplace = True)
# df_clif.drop(df_clif[df_clif['ID']=='ProstateX-0019_Finding1'].index, inplace = True)
# df_clif.drop(df_clif[df_clif['ID']=='ProstateX-0035_Finding1'].index, inplace = True)
# df_clif.drop(df_clif[df_clif['ID']=='ProstateX-0037_Finding1'].index, inplace = True)
# df_clif.drop(df_clif[df_clif['ID']=='ProstateX-0040_Finding3'].index, inplace = True)
# df_clif.drop(df_clif[df_clif['ID']=='ProstateX-0041_Finding1'].index, inplace = True)
# df_clif.drop(df_clif[df_clif['ID']=='ProstateX-0054_Finding1'].index, inplace = True)
# df_clif.drop(df_clif[df_clif['ID']=='ProstateX-0067_Finding1'].index, inplace = True)

df_clif.drop(df_clif [df_clif['ID']=='Finding2'].index, inplace=True)

In [135]:
df_clif.head(5) # Visualisation des données apres suppression

,ID,Clinically Significant,Gleason_Grade_Group
0,ProstateX-0000_Finding1,True,3
1,ProstateX-0001_Finding1,False,1
3,ProstateX-0002_Finding2,False,1
6,ProstateX-0004_Finding1,False,1
7,ProstateX-0005_Finding0,True,2


In [136]:
len(df_clif) # Nouvelle taille des données après supression des données ne disposant pas d'information clinique 

99

# Création des groupes en fonction du grades de Gleason

A ce niveau nous regroupons les données en fontion du grade de Gleason à partir du dataframe df_clif

In [137]:
df = df_clif

#Grade 1 
GG1_Grp = df.groupby(df.Gleason_Grade_Group)
GG1 = GG1_Grp.get_group('1')

#Grade 2
GG2_Grp = df.groupby(df.Gleason_Grade_Group)
GG2 = GG1_Grp.get_group('2')

# Grade 3
GG3_Grp = df.groupby(df.Gleason_Grade_Group)
GG3 = GG1_Grp.get_group('3')

#Grade 4
GG4_Grp = df.groupby(df.Gleason_Grade_Group)
GG4 = GG1_Grp.get_group('4')

#Grade 5
GG5_Grp = df.groupby(df.Gleason_Grade_Group)
GG5 = GG1_Grp.get_group('5')

# Extraction des ID des patiens

Nous aloons récupérer à ce niveau les les indexes ou les ID(Identifiant) de chaque patients en fonction des grades dans des fichiers différents(GG1.txt, GG2.txt, ...., GG5.txt). 

NB: Indiquer le chemin correspondant à l'emplacement du dossier dans lequel vous allez effectuer la sauvegarde

In [138]:
GG1_dic = []
GG2_dic = []
GG3_dic = []
GG4_dic = []
GG5_dic = []

for gg1 in GG1['ID']:
    GG1_dic.append(gg1)
    gg1 = gg1.split('_')[0]
    txt_file = open('lesions/Patient_ID/GG1.txt', 'a')
    txt_file.write(gg1 + "\n")
    
for gg2 in GG2['ID']:
    GG2_dic.append(gg2)
    gg2 = gg2.split('_')[0]
    txt_file = open('lesions/Patient_ID/GG2.txt', 'a')
    txt_file.write(gg2 + "\n")
    
for gg3 in GG3['ID']:
    GG3_dic.append(gg3)
    gg3 = gg3.split('_')[0]
    txt_file = open('lesions/Patient_ID/GG3.txt', 'a')
    txt_file.write(gg3 + "\n")
    
for gg4 in GG4['ID']:
    GG4_dic.append(gg4)
    gg4 = gg4.split('_')[0]
    txt_file = open('lesions/Patient_ID/GG4.txt', 'a')
    txt_file.write(gg4 + "\n")
    
for gg5 in GG5['ID']:
    GG5_dic.append(gg5)
    gg5 = gg5.split('_')[0]
    txt_file = open('lesions/Patient_ID/GG5.txt', 'a')
    txt_file.write(gg5 + "\n")

A ce niveau , nous créons des liste pour récupérer les identifiants des patients sauvegarder des les dictionnaires et fichiers précedement

In [139]:
GG1_indexes = []
GG2_indexes = []
GG3_indexes = []
GG4_indexes = []
GG5_indexes = []

for img in GG1_dic:
    GG1_indexes.append(img.split('-')[-1].split('_')[0])

for img in GG2_dic:
    GG2_indexes.append(img.split('-')[-1].split('_')[0])

for img in GG3_dic:
    GG3_indexes.append(img.split('-')[-1].split('_')[0])

for img in GG4_dic:
    GG4_indexes.append(img.split('-')[-1].split('_')[0])

for img in GG5_dic:
    GG5_indexes.append(img.split('-')[-1].split('_')[0])

# Pretraitment

A ce niveau , nous définissons des fonction nous permettant de normaliser les données des images et leus masques

In [69]:
def correctBiasField(img_input, mask):
   

    corrected = False
    img_output = np.zeros(img_input.shape)

    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    inputImage = sitk.GetImageFromArray(img_input)
    maskImage = sitk.GetImageFromArray((mask > 0.9).astype(np.uint8))
    inputImage = sitk.Cast(inputImage, sitk.sitkFloat32)
    output = corrector.Execute(inputImage, maskImage)
    img_output = sitk.GetArrayFromImage(output)
    corrected = True
      
    return img_output


def filterMedian(img_input, perc):
    
    img_flattened = img_input.flatten()

    th_low = np.percentile(img_flattened, perc)
    th_up = np.percentile(img_flattened, 100 - perc)

    mask_low = img_input < th_low
    mask_up = img_input > th_up

    img_output = img_input * (1 - mask_up) + th_up * mask_up
    img_output = img_output * (1 - mask_low) + th_low * mask_low

    return img_output


def normROI(img_input, mask):
    
    mask_binary = (mask > 0.9).astype(int)
    mask_nan = mask_binary / mask_binary
    img_masked_nan = img_input * mask_nan

    m = np.nanmean(img_masked_nan)
    s = np.nanstd(img_masked_nan)

    img_output = (img_input - m) * mask_binary / s

    return img_output

# Bias Field Correction

Nous appliquons la fonction précedente à nos données et les nouvelles données image générer sont sauvegarder pour être utiliser pour extraire les caractéristiques radiomiques.

NB: Les dossier sont déjà disponible dans le dossier lesion comme indiquer dans le Readme.md, il ne restera qu'à exécuter le code après les pré-requis indiquer dans le readme

# GG1

 * ADC

In [82]:
#Chemin vers nos données images et masques
root_dir = "lesions"

#Chemin de sauvegarde des données normalisés
save_root_dir = "lesions/Img_msk_resambled"

adc_igm_dir = os.path.join(root_dir,"Images/ADC")
adc_msk_dir = os.path.join(root_dir,"Masks/ADC")

GG1_img_save = os.path.join(save_root_dir, "GG1/Images_Resamble/ADC")

for index in GG1_indexes:
    adc_img = glob(adc_igm_dir + '/*' + index + '*')[0]
    adc_img_nii = nib.load(adc_img)
    img = adc_img_nii.get_fdata()
    print("img-"+index+" : ",adc_img_nii.shape)

    adc_msk = glob(adc_msk_dir + '/*' + index + '*')[0]
    adc_msk_nii = nib.load(adc_msk)
    mask = adc_msk_nii.get_fdata()
    print("msk-"+index+" : ",adc_msk_nii.shape)
    
    adc_img_bfc = correctBiasField(img, mask)
    adc_img_bfc_nii = nib.Nifti1Image(adc_img_bfc, adc_img_nii.affine)
    print("img_norm-"+index+" : ",adc_img_bfc_nii.shape)
    print('-------------------------')
    nib.save(adc_img_bfc_nii, os.path.join(GG1_img_save, ntpath.basename(adc_img)[:-17] + 'adc_preprocessed.nii'))
    
print('END of GG1')

END of GG1


 * T2

In [26]:
#Chemin vers nos données images et masques
root_dir = "lesions"

#Chemin de sauvegarde des données normalisés
save_root_dir = "lesions/Img_msk_resambled"

t2_igm_dir = os.path.join(root_dir,"Images/T2")
t2_msk_dir = os.path.join(root_dir,"Masks/T2")

GG1_img_save = os.path.join(save_root_dir, "GG1/Images_Resamble/T2")

for index in GG1_indexes:
    t2_img = glob(t2_igm_dir + '/*' + index + '*')[0]
    t2_img_nii = nib.load(t2_img)
    img = t2_img_nii.get_fdata()
    print("img-"+index+" : ",t2_img_nii.shape)

    t2_msk = glob(t2_msk_dir + '/*' + index + '*')[0]
    t2_msk_nii = nib.load(t2_msk)
    mask = t2_msk_nii.get_fdata()
    print("msk-"+index+" : ",t2_msk_nii.shape)
    
    t2_img_bfc = correctBiasField(img, mask)
    t2_img_bfc_nii = nib.Nifti1Image(t2_img_bfc, t2_img_nii.affine)
    print("img_norm-"+index+" : ",t2_img_bfc_nii.shape)
    print('-------------------------')
    nib.save(t2_img_bfc_nii, os.path.join(GG1_img_save, ntpath.basename(t2_img)[:-14] + '_preprocessed.nii'))
    
print('END of GG1')

END of GG1


GG2

* ADC

In [155]:
#Chemin vers nos données images et masques
root_dir = "lesions"

#Chemin de sauvegarde des données normalisés
save_root_dir = "lesions/Img_msk_resambled"

adc_igm_dir = os.path.join(root_dir,"Images/ADC")
adc_msk_dir = os.path.join(root_dir,"Masks/ADC")

GG2_img_save = os.path.join(save_root_dir, "GG2/Images_Resamble/ADC")

for index in GG2_indexes:
    adc_img = glob(adc_igm_dir + '/*' + index + '*')[0]
    adc_img_nii = nib.load(adc_img)
    img = adc_img_nii.get_fdata()
    print("img-"+index+" : ",adc_img_nii.shape)

    adc_msk = glob(adc_msk_dir + '/*' + index + '*')[0]
    adc_msk_nii = nib.load(adc_msk)
    mask = adc_msk_nii.get_fdata()
    print("msk-"+index+" : ",adc_msk_nii.shape)
    
    adc_img_bfc = correctBiasField(img, mask)
    adc_img_bfc_nii = nib.Nifti1Image(adc_img_bfc, adc_img_nii.affine)
    print("img_norm-"+index+" : ",adc_img_bfc_nii.shape)
    print('-------------------------')
    nib.save(adc_img_bfc_nii, os.path.join(GG2_img_save, ntpath.basename(adc_img)[:-17] + 'adc_preprocessed.nii'))
    
print('END of GG2')

* T2

In [154]:
#Chemin vers nos données images et masques
root_dir = "lesions"

#Chemin de sauvegarde des données normalisés
save_root_dir = "lesions/Img_msk_resambled"

t2_igm_dir = os.path.join(root_dir,"Images/T2")
t2_msk_dir = os.path.join(root_dir,"Masks/T2")

GG2_img_save = os.path.join(save_root_dir, "GG2/Images_Resamble/T2")

for index in GG2_indexes:
    t2_img = glob(t2_igm_dir + '/*' + index + '*')[0]
    t2_img_nii = nib.load(t2_img)
    img = t2_img_nii.get_fdata()
    print("img-"+index+" : ",t2_img_nii.shape)
    
    t2_msk = glob(t2_msk_dir + '/*' + index + '*')[0]
    t2_msk_nii = nib.load(t2_msk)
    mask = t2_msk_nii.get_fdata()
    print("msk-"+index+" : ",t2_msk_nii.shape)
    
    t2_img_bfc = correctBiasField(img, mask)
    t2_img_bfc_nii = nib.Nifti1Image(t2_img_bfc, t2_img_nii.affine)
    print("img_norm-"+index+" : ",t2_img_bfc_nii.shape)
    print('-------------------------')
    nib.save(t2_img_bfc_nii, os.path.join(GG2_img_save, ntpath.basename(t2_img)[:-14] + '_preprocessed.nii'))
    
print('END of GG2')

GG3

* ADC

In [23]:
#Chemin vers nos données images et masques
root_dir = "lesions"

#Chemin de sauvegarde des données normalisés
save_root_dir = "lesions/Img_msk_resambled"

adc_igm_dir = os.path.join(root_dir,"Images/ADC")
adc_msk_dir = os.path.join(root_dir,"Masks/ADC")

GG3_img_save = os.path.join(save_root_dir, "GG3/Images_Resamble/ADC")

for index in GG3_indexes:
    adc_img = glob(adc_igm_dir + '/*' + index + '*')[0]
    adc_img_nii = nib.load(adc_img)
    img = adc_img_nii.get_fdata()
    print("img-"+index+" : ",adc_img_nii.shape)

    adc_msk = glob(adc_msk_dir + '/*' + index + '*')[0]
    adc_msk_nii = nib.load(adc_msk)
    mask = adc_msk_nii.get_fdata()
    print("msk-"+index+" : ",adc_msk_nii.shape)
    
    adc_img_bfc = correctBiasField(img, mask)
    adc_img_bfc_nii = nib.Nifti1Image(adc_img_bfc, adc_img_nii.affine)
    print("img_norm-"+index+" : ",adc_img_bfc_nii.shape)
    print('-------------------------')
    nib.save(adc_img_bfc_nii, os.path.join(GG3_img_save, ntpath.basename(adc_img)[:-17] + 'adc_preprocessed.nii'))
    
print('END of GG3')

END of GG3


* T2

In [28]:
#Chemin vers nos données images et masques
root_dir = "lesions"

#Chemin de sauvegarde des données normalisés
save_root_dir = "lesions/Img_msk_resambled"

t2_igm_dir = os.path.join(root_dir,"Images/T2")
t2_msk_dir = os.path.join(root_dir,"Masks/T2")

GG3_img_save = os.path.join(save_root_dir, "GG3/Images_Resamble/T2")

for index in GG3_indexes:
    t2_img = glob(t2_igm_dir + '/*' + index + '*')[0]
    t2_img_nii = nib.load(t2_img)
    img = t2_img_nii.get_fdata()
    print("img-"+index+" : ",t2_img_nii.shape)
    
    t2_msk = glob(t2_msk_dir + '/*' + index + '*')[0]
    t2_msk_nii = nib.load(t2_msk)
    mask = t2_msk_nii.get_fdata()
    print("msk-"+index+" : ",t2_msk_nii.shape)
    
    t2_img_bfc = correctBiasField(img, mask)
    t2_img_bfc_nii = nib.Nifti1Image(t2_img_bfc, t2_img_nii.affine)
    print("img_norm-"+index+" : ",t2_img_bfc_nii.shape)
    print('-------------------------')
    nib.save(t2_img_bfc_nii, os.path.join(GG3_img_save, ntpath.basename(t2_img)[:-14] + '_preprocessed.nii'))
    
print('END of GG3')

END of GG3


In [ ]:
GG4

* ADC

In [52]:
#Chemin vers nos données images et masques
root_dir = "lesions"

#Chemin de sauvegarde des données normalisés
save_root_dir = "lesions/Img_msk_resambled"

adc_igm_dir = os.path.join(root_dir,"Images/ADC")
adc_msk_dir = os.path.join(root_dir,"Masks/ADC")

GG4_img_save = os.path.join(save_root_dir, "GG4/Images_Resamble/ADC")

for index in GG4_indexes:
    adc_img = glob(adc_igm_dir + '/*' + index + '*')[0]
    adc_img_nii = nib.load(adc_img)
    img = adc_img_nii.get_fdata()
    print("img-"+index+" : ",adc_img_nii.shape)

    adc_msk = glob(adc_msk_dir + '/*' + index + '*')[0]
    adc_msk_nii = nib.load(adc_msk)
    mask = adc_msk_nii.get_fdata()
    print("msk-"+index+" : ",adc_msk_nii.shape)
    
    adc_img_bfc = correctBiasField(img, mask)
    adc_img_bfc_nii = nib.Nifti1Image(adc_img_bfc, adc_img_nii.affine)
    print("img_norm-"+index+" : ",adc_img_bfc_nii.shape)
    print('-------------------------')
    nib.save(adc_img_bfc_nii, os.path.join(GG4_img_save, ntpath.basename(adc_img)[:-17] + 'adc_preprocessed.nii'))
    
print('END of GG4')

END of GG4


* T2

In [29]:
#Chemin vers nos données images et masques
root_dir = "lesions"

#Chemin de sauvegarde des données normalisés
save_root_dir = "lesions/Img_msk_resambled"

t2_igm_dir = os.path.join(root_dir,"Images/T2")
t2_msk_dir = os.path.join(root_dir,"Masks/T2")

GG4_img_save = os.path.join(save_root_dir, "GG4/Images_Resamble/T2")

for index in GG4_indexes:
    t2_img = glob(t2_igm_dir + '/*' + index + '*')[0]
    t2_img_nii = nib.load(t2_img)
    img = t2_img_nii.get_fdata()
    print("img-"+index+" : ",t2_img_nii.shape)

    t2_msk = glob(t2_msk_dir + '/*' + index + '*')[0]
    t2_msk_nii = nib.load(t2_msk)
    mask = t2_msk_nii.get_fdata()
    print("msk-"+index+" : ",t2_msk_nii.shape)
    
    t2_img_bfc = correctBiasField(img, mask)
    t2_img_bfc_nii = nib.Nifti1Image(t2_img_bfc, t2_img_nii.affine)
    print("img_norm-"+index+" : ",t2_img_bfc_nii.shape)
    print('-------------------------')
    nib.save(t2_img_bfc_nii, os.path.join(GG4_img_save, ntpath.basename(t2_img)[:-14] + '_preprocessed.nii'))
    
print('END of GG4')

END of GG4


In [ ]:
GG5

* ADC

In [35]:
#Chemin vers nos données images et masques
root_dir = "lesions"

#Chemin de sauvegarde des données normalisés
save_root_dir = "lesions/Img_msk_resambled"

adc_igm_dir = os.path.join(root_dir,"Images/ADC")
adc_msk_dir = os.path.join(root_dir,"Masks/ADC")

GG5_img_save = os.path.join(save_root_dir, "GG5/Images_Resamble/ADC")

for index in GG5_indexes:
    adc_img = glob(adc_igm_dir + '/*' + index + '*')[0]
    adc_img_nii = nib.load(adc_img)
    img = adc_img_nii.get_fdata()
    print("img-"+index+" : ",adc_img_nii.shape)

    adc_msk = glob(adc_msk_dir + '/*' + index + '*')[0]
    adc_msk_nii = nib.load(adc_msk)
    mask = adc_msk_nii.get_fdata()
    print("msk-"+index+" : ",adc_msk_nii.shape)
    
    adc_img_bfc = correctBiasField(img, mask)
    adc_img_bfc_nii = nib.Nifti1Image(adc_img_bfc, adc_img_nii.affine)
    print("img_norm-"+index+" : ",adc_img_bfc_nii.shape)
    print('-------------------------')
    nib.save(adc_img_bfc_nii, os.path.join(GG5_img_save, ntpath.basename(adc_img)[:-17] + 'adc_preprocessed.nii'))
    
print('END of GG5')

END of GG5


* T2

In [37]:
#Chemin vers nos données images et masques
root_dir = "lesions"

#Chemin de sauvegarde des données normalisés
save_root_dir = "lesions/Img_msk_resambled"

t2_igm_dir = os.path.join(root_dir,"Images/T2")
t2_msk_dir = os.path.join(root_dir,"Masks/T2")

GG5_img_save = os.path.join(save_root_dir, "GG5/Images_Resamble/T2")

for index in GG5_indexes:
    t2_img = glob(t2_igm_dir + '/*' + index + '*')[0]
    t2_img_nii = nib.load(t2_img)
    img = t2_img_nii.get_fdata()
    print("img-"+index+" : ",t2_img_nii.shape)

    t2_msk = glob(t2_msk_dir + '/*' + index + '*')[0]
    t2_msk_nii = nib.load(t2_msk)
    mask = t2_msk_nii.get_fdata()
    print("msk-"+index+" : ",t2_msk_nii.shape)
    
    
    t2_img_bfc = correctBiasField(img, mask)
    t2_img_bfc_nii = nib.Nifti1Image(t2_img_bfc, t2_img_nii.affine)
    print("img_norm-"+index+" : ",t2_img_bfc_nii.shape)
    print('-------------------------')
    nib.save(t2_img_bfc_nii, os.path.join(GG5_img_save, ntpath.basename(t2_img)[:-14] + '_preprocessed.nii'))
    
print('END of GG5')

END of GG5


# Features extractions

Dans cette partie nous allons extraire les caractéristiques radiomique en fonctions des grades 

In [114]:
def extractFeatures(img_input, mask):

    i = sitk.GetImageFromArray(img_input) #Chargement des images et masques sous forme matriciel
    m = sitk.GetImageFromArray(mask)
    f_v = {}  # Initialisation de dictionnaire

    #Création de dictionnaire d'initialisation
    settings = {}
    settings['preCrop'] = True
    settings['normalize'] = False
    settings['binCount'] = 50
    settings['resampledPixelSpacing'] = None
    settings['interpolator'] = sitk.sitkBSpline

    #Variable d'extraction de features radiomics à partir de featureextractor de la bibliothèque PyRadiomic
    extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
    
    extractor.addProvenance(False)
    extractor.disableAllFeatures()
    extractor.enableImageTypes(Original={})

    # Définition des différentes caractéristiques à extraire
    extractor.enableFeatureClassByName('firstorder', enabled=True)
    extractor.enableFeatureClassByName('shape', enabled=True)
    extractor.enableFeatureClassByName('glcm', enabled=True)
    extractor.enableFeatureClassByName('glrlm', enabled=True)
    extractor.enableFeatureClassByName('glszm', enabled=True)
    extractor.enableFeatureClassByName('gldm', enabled=True)
    extractor.enableFeatureClassByName('ngtdm', enabled=True)

    # Passage de l'image et du masque pour l'extraction
    extracted_ftrs = extractor.execute(i, m)

    # Définition de la clef et valeur pour les caractéristiques à extraire
    for key, value in extracted_ftrs.items():
        f_v[key] = value

    return f_v

In [115]:
# Cette fontion permet de créer les en-tête des colonnes

def createHeader():
    
    f = {'sub': ''}
    i_t = np.ones((2, 2, 2))
    m_t = np.zeros((2, 2, 2))
    m_t[0, 0, 0] = 1
    m_t[1, 1, 1] = 1
    f.update(extractFeatures(i_t, m_t))

    return list(f.keys())

In [116]:
#Fonction d'extraction proprement dite

def extractFromeTissuesThroughSubFolders(ds_folder, ds_names, 
                                         adc_modality = 'ADC', 
                                         t2_modality = 'T2'):
    
    gg = ds_folder.split('/')[-2]
    
    h = createHeader()
    
    # Dossier pour sauvegarder les caractéristiques extraits
    root_dir = 'lesions/ADC_T2_Images_Analyses/ADC_T2_features/'
    
    mask_dir = 'lesions/'
    
    #ouverture du fichier avec option ajout 'a' si le fichier n'existe pas
    adc_csv_file = open(root_dir  + gg + '_' + adc_modality + '_features.csv' , 'a')
    
    #Ecriture des caractéristiquex extraits dans le fichier créer
    adc_writer = csv.DictWriter(adc_csv_file, fieldnames = h, lineterminator='\n')
    adc_writer.writeheader()
    
    #ouverture du fichier avec option ajout 'a' si le fichier n'existe pas
    t2_csv_file = open(root_dir + gg + '_' + t2_modality + '_features.csv' , 'a')
    
    #Ecriture des caractéristiquex extraits dans le fichier créer
    t2_writer = csv.DictWriter(t2_csv_file, fieldnames = h, lineterminator='\n')
    t2_writer.writeheader()
    
    with tqdm(total=len(ds_names), file=sys.stdout) as pbar:        
        for sub in ds_names: #ds_name représente les indexes des patients regrouper précédement en fonction des grades

            #Chargement des images normalisées ADC  
            adc_img_path = ds_folder + "Images_Resamble/" + adc_modality + '/ProstateX-' + sub +'_ep2d_adc_preprocessed.nii'
            adc_img_nii = nib.load(adc_img_path)
            adc_img =adc_img_nii.get_fdata()
    
            # Chargement des images normalisées T2  
            t2_img_path = ds_folder + "Images_Resamble/" + t2_modality + '/ProstateX-' + sub +'_preprocessed.nii'
            t2_img_nii = nib.load(t2_img_path)
            t2_img =t2_img_nii.get_fdata()
                
            # Chargement des masques ADC pour chaque image. C'est l'importances des indexes d'images qui nous permettent
            # d'utiliser pour chaque image le masque correspondant
            adc_roi_path = mask_dir + 'Masks/'  + adc_modality + '/ProstateX-' + sub + '-Finding_ADC.nii.gz'                         
            adc_roi_nii = nib.load(adc_roi_path)
            adc_roi = adc_roi_nii.get_fdata()
            
            # Chargement des masques T2
            t2_roi_path = mask_dir + 'Masks/'  + t2_modality + '/ProstateX-' + sub + '-Finding-t2_ROI.nii.gz'                         
            t2_roi_nii = nib.load(t2_roi_path)
            t2_roi = t2_roi_nii.get_fdata()
            
            adc_img_preproc = np.copy(adc_img)
            t2_img_preproc = np.copy(t2_img)
            
            # Création de la colonne Sub à qui on affecte l'index qui représente l'ID du patient
            # précéder de ProstateX
            f_sub_adc = {'sub': 'ProstateX_' + str(sub)}
            # Association du f_sub_adc créer aux carractéristiques extraits pour chaque patient
            f_sub_adc.update(extractFeatures(adc_img_preproc, adc_roi))
            adc_writer.writerow(f_sub_adc)
            
            f_sub_t2 = {'sub': 'ProstateX_' + str(sub)}
            f_sub_t2.update(extractFeatures(t2_img_preproc, t2_roi))
            t2_writer.writerow(f_sub_t2)

            pbar.set_description('processed: %s' % sub)
            pbar.update(1)


    adc_csv_file.close()
    t2_csv_file.close()

In [117]:
root = 'lesions/Img_msk_resambled'

ds1_folder = os.path.join(root, 'GG1/')
ds2_folder = os.path.join(root, 'GG2/')
ds3_folder = os.path.join(root, 'GG3/')
ds4_folder = os.path.join(root, 'GG4/')
ds5_folder = os.path.join(root, 'GG5/')

J'ai deux modalités d'images à extraire

* Modalité T2

* Modalité ADC (Diffusion)

# GG1

In [147]:
extractFromeTissuesThroughSubFolders( ds1_folder, GG1_indexes)

# GG2

In [148]:
extractFromeTissuesThroughSubFolders( ds2_folder, GG2_indexes)

# GG3

In [149]:
extractFromeTissuesThroughSubFolders( ds3_folder, GG3_indexes)

# GG4

In [150]:
extractFromeTissuesThroughSubFolders( ds4_folder, GG4_indexes)

# GG5

In [153]:
extractFromeTissuesThroughSubFolders( ds5_folder, GG5_indexes)